In [1]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as numpy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime
import time

def lazyloading(driver):
    # Get the current number of rows
    current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
    total_number = int(driver.find_elements_by_css_selector('.querySummary__querySummary--39WP2')[0].text.split('Showing ')[1].split('Rosé')[0])
    print("Total Number: ", total_number)
    while current_rows_number < total_number:
        # Scroll down to make new XHR (request more table rows)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        try:
            driver.set_window_position(0,-1000)
            # Wait until number of rows increased 
            time.sleep(10)
            wait(driver, 200).until(lambda drive: len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0')) > current_rows_number)
            # Update variable with current rows number
            current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
            if current_rows_number % 200 == 0:
              print("Curr Row Number: ", current_rows_number)
        # If number of rows remains the same after 5 seconds passed, break the loop
        # as there no more rows to receive
        except TimeoutException:
            break

In [ ]:
from wine_setup import WineSetup

rose_wine_url = "https://www.vivino.com/explore?e=eJwtyzEOwjAQRNHbTBklCpTT5QaICiG0cRbLEnaQ1wokp48LN_NfMzFz7K6IIXHoEeVf08PtvN_g6kz4coB_c5MctMgH68wsJSRvL9k0i1esXNQcfuXx5KWerVatIZSGQxrSAaOTE7oKKeU="
rose_wine = Wine(rose_wine_url)
rose_wine.setup_driver()
rose_wine.run_timestamp()
lazyloading(rose_wine.driver)
rose_wine.run_timestamp()

Current Timestamp:  01-Feb-2020 (20:32:43.942533)
Total Number:  410
Curr Row Number:  200
Curr Row Number:  400


In [50]:
rose_html = BeautifulSoup(rose_wine.driver.page_source, 'lxml')
rose_div = rose_html.find("div", {"class": "explorerPage__results--3wqLw"})
rose_rows = rose_html.find_all("div", {"class": "explorerCard__explorerCard--3Q7_0"})

In [51]:
import copy

# austria_wine_df = pd.DataFrame()
all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "title": None, "location": None, "country": None, "price": None, "type": None, "ratings": None, "num_ratings": None, "reviews": None, "url": None
}

for row in rose_rows:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    new_row['title'] = row.find("span", {"class": "vintageTitle__wine--U7t9G"}).text
    location = row.find("div", {"class": "vintageLocation__vintageLocation--1DF0p"})
    new_row['location'] = location.findChildren()[-1].text
    country = row.find("i", {"class": "vintageLocation__countryFlag--1HbXr"})['title']
    new_row['country'] = country
    price_button = row.find("button", {"class": "addToCartButton__addToCartButton--qZv9F"})
    if price_button:
        new_row['price'] = (float(price_button.find("span").text.replace("$", "")))
    new_row['type'] = 'Rosé'
    new_row['ratings'] = row.find("div", {"class": "vivinoRatingWide__averageValue--1zL_5"}).text
    new_row['num_ratings'] = int(row.find("div", {"class": "vivinoRatingWide__basedOn--s6y0t"}).text.split()[0])
    review_div = row.find("div", {"class": "review__note--2b2DB"})
    if review_div:
        new_row['reviews'] = review_div.text
    clean_div = row.find("div", {"class": "cleanWineCard__cleanWineCard--tzKxV cleanWineCard__row--CBPRR"})
    if clean_div:
        new_row['url'] = 'https://www.vivino.com' + clean_div.find("a")['href']
    all_rows.append(new_row)

In [52]:
all_rows

[{'title': 'En Valingrain Rosé des Riceys 2014',
  'location': 'Rosé des Riceys',
  'region': 'Champagne',
  'country': 'France',
  'price': None,
  'type': 'Rosé',
  'ratings': '3.9',
  'num_ratings': 25,
  'reviews': None,
  'image_url': 'https://ct.pinterest.com/v3/?event=init&tid=2612573710865&noscript=1'},
 {'title': 'En Barmont Rosé des Riceys 2011',
  'location': 'Rosé des Riceys',
  'region': 'Champagne',
  'country': 'France',
  'price': None,
  'type': 'Rosé',
  'ratings': '3.9',
  'num_ratings': 25,
  'reviews': None,
  'image_url': 'https://ct.pinterest.com/v3/?event=init&tid=2612573710865&noscript=1'},
 {'title': 'En Valingrain Rosé des Riceys 2010',
  'location': 'Rosé des Riceys',
  'region': 'Champagne',
  'country': 'France',
  'price': None,
  'type': 'Rosé',
  'ratings': '3.8',
  'num_ratings': 51,
  'reviews': None,
  'image_url': 'https://ct.pinterest.com/v3/?event=init&tid=2612573710865&noscript=1'},
 {'title': 'En Barmont Rosé des Riceys 2012',
  'location': 'Ros

In [14]:
conda install pymongo

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [21]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
wine_db = client['wine_db']
wines = wine_db['wines']

In [53]:
for row in all_rows:
    wines.insert_one(row)

In [ ]:
for row in all_rows:
    wines.insert_one(row)